In [1]:
def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

In [7]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request

task1='emotion'
MODEL1 = f"cardiffnlp/twitter-roberta-base-{task1}"
model1 = TFAutoModelForSequenceClassification.from_pretrained(MODEL1)
tokenizer1 = AutoTokenizer.from_pretrained(MODEL1)

task2='irony'
MODEL2 = f"cardiffnlp/twitter-roberta-base-{task2}"
model2 = TFAutoModelForSequenceClassification.from_pretrained(MODEL2)
tokenizer2 = AutoTokenizer.from_pretrained(MODEL2)



All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-emotion.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


Downloading:   0%|          | 0.00/589 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-irony.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [26]:
# download label mapping emotion
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task1}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels1 = [row[1] for row in csvreader if len(row) > 1]
labels1

['anger', 'joy', 'optimism', 'sadness']

In [27]:
# download label mapping irony
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task2}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels2 = [row[1] for row in csvreader if len(row) > 1]
labels2

['non_irony', 'irony']

In [46]:
# PT
#model = AutoModelForSequenceClassification.from_pretrained(MODEL)

#text = "Good night 😊"
#text = preprocess(text)
#encoded_input = tokenizer(text, return_tensors='pt')
#output = model(**encoded_input)
#scores = output[0][0].detach().numpy()
#scores = softmax(scores)

# # TF
text1 = "Good night 😊"
encoded_input1 = tokenizer1(text1, return_tensors='tf')
output1 = model1(encoded_input)
scores1 = output[0][0].numpy()
scores1 = softmax(scores)

In [47]:
ranking1 = np.argsort(scores1)
ranking1 = ranking1[::-1]
for i in range(scores1.shape[0]):
    l1 = labels1[ranking1[i]]
    s1 = scores1[ranking1[i]]
    print(f"{i+1}) {l1} {np.round(float(s1), 4)}")

1) joy 0.3622
2) sadness 0.2488
3) optimism 0.1955
4) anger 0.1935


In [48]:
text2 = "Good night #noooot"
encoded_input2 = tokenizer2(text2, return_tensors='tf')
output2 = model2(encoded_input)
scores2 = output[0][0].numpy()
scores2 = softmax(scores)

In [49]:
ranking2 = np.argsort(scores2)
ranking2 = ranking2[::-1]
for i in range(scores2.shape[0]):
    l2 = labels2[ranking2[i]]
    s2 = scores2[ranking2[i]]
    print(f"{i+1}) {l2} {np.round(float(s2), 4)}")

1) irony 0.3622


IndexError: list index out of range

1) irony 0.6547


IndexError: list index out of range